# **Welcome to ABDICO_parsing Code Notebook**

It is often useful to extract the main entities and actions described in a document. For example, this is useful in the Institutional Grammar (IG) for identifying some of the most common parts of an institutional statement, like the Attribute and Object, along with their Aim and Deontic.

This notebook uses new methods to automate this extraction process, giving policy scholars an easy route to some of the most common IG elements (though it's not restricted to IG, and is useful for extracting (and relating) these syntactic/semantic elements in any kind of text).

### **This notebook performs the following tasks**


*   It takes a corpus of policy sentences from any domain and extracts four central components if the institutional grammar from each: Attribute, Aim, Deontic, and Object
  *   By default, it performs these tasks on a dataset we provide, but the intent is that users will upload their own datasets.
*   When given unlabeled data, it applies labels
*   When given labeled (such as hand-coded) data, it applies labels and compares the accuracy of its labels to those provided.
*   It outputs a downloadable file of the original statements with labels, as well as statistics and figures that can be used in reporting.



### This notebook follows an approach to automatic coding with several features

Our approach is distinct from other manual and automatic approach to IG coding:
* *It requires no training data*. You don't have to do any labelling. Labels may still be useful if you want a sense of the accuracy of our approach for your method, but such validation requires the coding of only a sample.
* *It supports users of many experience levels*. To benefit from our approach, at minimum you have to be able to get your data into the right format (the right number of spreadsheet columns in the right order), upload it, run the code, and download the outputs. You don't need to know how to program, and you don't have to understand this notebook, you just have to be bold enough to face it.  But because we expose the code, people who know how to code or are trying to learn can change it and adapt it.
* *It uses both syntactic and semantic information*. This is important because not any grammatical subject of a sentence (a syntactic property) can be an Attribute, that subject has to be some kind of agent capable of action and decision making (semantic property)
* *It is naturally robust to passive voice*. Passive voice increases the complexity of manual coding a bit, and automatic coding a lot.  Our approach works around this issue to naturally identify Attributes and Objects even when they are grammatical objects and subjects, respectively.
* *It gets better as AI gets better*. This is because the code is written to permit the switching-in of different models as "back-ends" for the labelling.
* *With minor changes it can accommodate other languages*.  For the same reason as above, if a back-end model has been written for your language you can code in that language.  
* *We do not implement the full institutional grammar* A drawback of our approach is that it only extracts the four most commonly used part of IG. A related drawback is that there are minor technical differences in how we define those entities (such as agent) and how formal IG defines them. Our operationalization is close enough that it is sufficient to be clear in reporting about where your labels came from.



# **Installation & Setup**
This code sets up the analysis. You don't have to understand it. Just run it and then scroll down.

Run these commands below for necessary installations. To run press ***ctrl+enter*** keys.

In [1]:
!git clone https://github.com/BSAkash/NLP4GOV
%cd NLP4GOV
!pip install -q -r ./SRL/requirements.txt
!python -m spacy download en_core_web_sm
from SRL import SRL
%cd /content/

Cloning into 'NLP4GOV'...
remote: Enumerating objects: 2145, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 2145 (delta 158), reused 280 (delta 139), pack-reused 1844
Receiving objects: 100% (2145/2145), 7.94 MiB | 13.67 MiB/s, done.
Resolving deltas: 100% (1257/1257), done.
/content/NLP4GOV
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 30.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirem

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


...

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/content


# **Data upload**
Upload your dataset file here. Or, to see it run on an example dataset, download the provided example datafile [ASF_policies_coref_resolved.csv](https://storage.googleapis.com/public_data_c2/IG_datasets/ASF_policies_coref_resolved.csv) , to see example outputs and to become familiar with the flow.

For your own data, you will likely have to adapt it for this notebook to run. See below for the sample format

First run your own data `.csv` file through [ABDICO_coreferences.ipynb](https://github.com/BSAkash/NLP4GOV/blob/master/ABDICO_coreferences.ipynb) file. Download the coreferences resolved `main.csv` file from there and upload that file here as `main.csv` in a `.csv` file format and Policy document column should be named `raw institutional statement`.
* ### NOTE
    - It is ideal to limit your data with 5-10 sentences per each row.

In [2]:
##allow user to upload files : Upload file should be named main.csv. Policy statement column should be named "raw institutional statement"
import os
from google.colab import files
uploaded = files.upload()
os.rename(list(uploaded.keys())[0], 'main.csv')

This is our ASF policy dataset for demonstration purpose and you do not need to run this if you have uploaded your own file in the above cell. If you still want to run this please make sure to uncomment the code in the below cell using the (ctrl+/) keys

In [3]:
# !wget -O main.csv https://storage.googleapis.com/public_data_c2/IG_datasets/ASF_policies_coref_resolved.csv

--2024-03-21 07:38:52--  https://storage.googleapis.com/public_data_c2/IG_datasets/ASF_policies_coref_resolved.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c06::cf, 2607:f8b0:4023:c0d::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209611 (205K) [text/csv]
Saving to: ‘main.csv’

main.csv            100%[===================>] 204.70K  --.-KB/s    in 0.02s   

2024-03-21 07:38:53 (8.75 MB/s) - ‘main.csv’ saved [209611/209611]




# Applying labels

*  Run this cell to get ABDICO constiuents on "main.csv" list item
* The `.inference()` function below processes your uploaded dataset to add labels based on the text column.


In [4]:
%cd /content
instance = SRL.SRL()
instance.inference('main.csv')

/content


# **Download labeled data**
* Download to write the data to a file and retreive labeled data in that file.

* Download is in `.csv` format and can be opened in any spreadsheet software.

* Outputs can be found under "Attribute inf", "Object inf", "Deontic inf", "Aim inf"

In [5]:
files.download('main.csv')# Download the file and view the results here!!

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import pandas as pd
df = pd.read_csv('main.csv')
df

,policy.doc.name,section.name,document,coref,raw institutional statement,sentences,ROOT,srl_ip,srl_parsed,srl_verb,arg_len,keep,attribute_inf,object_inf,aim_inf,deontic_inf
0,graduation guide,graduation approval vote,once the accepting tlp have voted to accept th...,once the accepting tlp have voted to accept th...,once the accepting tlp have voted to accept th...,once the accepting tlp have voted to accept th...,sent,[{'sentence': 'once the accepting tlp have vot...,"{'ARGM-MOD': ['should'], 'ARGM-MNR': ['via gen...",sent,7,True,notice,to the ipmc,sent,should
1,graduation guide,the recommendation vote,"once a consensus has been reached, a vote shou...","once a consensus has been reached , a vote sho...","once a consensus has been reached , a vote sho...","once a consensus has been reached , a vote sho...",started,[{'sentence': 'once a consensus has been reach...,"{'ARGM-MOD': ['should'], 'ARGM-LOC': ['on the ...",started,7,True,by a member of the ppmc,a vote,started,should
2,ppmc guide,adding committers,if the nominee is already an apache committer ...,if the nominee is already an apache committer ...,if the nominee is already an apache committer ...,if the nominee is already an apache committer ...,add,[{'sentence': 'if the nominee is already an ap...,"{'ARGM-MOD': ['can'], 'ARGM-MNR': ['via the wh...",add,7,True,the ipmc member,the nominee,add,can
3,incubator policy,joining the ipmc,"after a vote has finished, the ipmc must send ...","after a vote has finished , the ipmc must send...","after a vote has finished , the ipmc must send...","after a vote has finished , the ipmc must send...",send,"[{'sentence': 'after a vote has finished , the...","{'ARGM-MOD': ['must'], 'ARGM-DIR': ['email'], ...",send,7,True,the ipmc,a notice,send,must
4,incubator policy,adding ppmc members,after voting someone into the ppmc on the ppmc...,after voting someone into the ppmc on the ppmc...,after voting someone into the ppmc on the ppmc...,after voting someone into the ppmc on the ppmc...,send,[{'sentence': 'after voting someone into the p...,"{'ARGM-MOD': ['must'], 'ARGM-DIR': ['email'], ...",send,7,True,the podling,a notice,send,must
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,graduation guide,creating an open and diverse community,a major criterion for graduation is to have de...,a major criterion for graduation is to have de...,a major criterion for graduation is to have de...,a major criterion for graduation is to have de...,developed,[{'sentence': 'a major criterion for graduatio...,{'ARG1': ['an open and diverse meritocratic co...,developed,2,True,NaN,an open and diverse meritocratic community,developed,NaN
260,ppmc guide,voting in a new ppmc member,note that there is a grace period of 72 hours ...,note that there is a grace period of 72 hours ...,note that there is a grace period of 72 hours ...,note that there is a grace period of 72 hours ...,note,[{'sentence': 'note that there is a grace peri...,{'ARG1': ['that there is a grace period of 72 ...,note,2,True,NaN,that there is a grace period of 72 hours from ...,note,NaN
261,incubator policy,website,there are restrictions on where podlings can h...,there are restrictions on where podlings can h...,there are restrictions on where podlings can h...,there are restrictions on where podlings can h...,are,"[{'sentence': ""there are restrictions on where...","{'ARG1': [""restrictions on where podlings can ...",are,2,True,NaN,restrictions on where podlings can host podlin...,are,NaN
262,incubator policy,branding,there are restrictions about how podlings can ...,there are restrictions about how podlings can ...,there are restrictions about how podlings can ...,there are restrictions about how podlings can ...,are,"[{'sentence': ""there are restrictions about ho...","{'ARG1': [""restrictions about how podlings can...",are,2,True,NaN,restrictions about how podlings can use podlin...,are,NaN


# **Extra: Calculate accuracy of labels compared to pre-existing labels**
If the uploaded data has columns for labels that were applied by another method (e.g. manually), this code will compare the computed labels to the pre-existing labels. Make sure that the originally uploaded file has the following format:
* Please name uploaded file as `main.csv` in a `.csv` file format and Policy document column should be named `raw institutional statement`.

* That is the format reflected in the sample data we provide with this notebook.

The statistic we provide a is typical useful summary statistic for classifiers of this type. Specifically, we provide F1, an alternative to accuracy that is suitable for evaluating a classifier when its labels are rare. Like accuracy, F1 ranges from 0 to 1. Read more about F1 score [here](https://towardsdatascience.com/the-f1-score-bec2bbc38aa6)



* Run this cell to check performance of the notebook on other datasets
* The box plots below illustrate the range of F1 scores for each type of detected element.
* Any contributions to the dataset can be viewed in the evaluation results below which includes several datasets along with yours!!

In [7]:
%cd NLP4GOV
evaluation = SRL.SRL("eval")
plt = evaluation.srl_eval()

/content/NLP4GOV
Aquaculture_Policies Dataset:  156
Dataset after removing uncoded statements:  153
Abstractive coding in one or more fields: 152
 F1 score for attribute: 0.7603353601055807
 F1 score for object: 0.5682564860150076
 F1 score for deontic: 0.935251798561151
 F1 score for aim: 0.8734265734265735
Food_policy_data Dataset:  496
Dataset after removing uncoded statements:  398
 F1 score for attribute: 0.7161072932477918
 F1 score for object: 0.5756398444937255
 F1 score for deontic: 0.9367088607594937
 F1 score for aim: 0.8329113924050633
National_Organic_Program Dataset:  1078
Dataset after removing uncoded statements:  835
Abstractive coding in one or more fields: 731
 F1 score for attribute: 0.8615659833082765
 F1 score for object: 0.4911541463005999
 F1 score for deontic: 0.9348617176128093
 F1 score for aim: 0.8359404096834264
Mean F1 score for attribute: 0.7700092341042162
Mean F1 score for object: 0.5297632761119571
Mean F1 score for deontic: 0.9356518817204301
Mean F1 